<a href="https://colab.research.google.com/github/mintra-tippayaratsontorn/data_viz_2022/blob/main/DataPrepeocessing_18_03_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%cd /content/drive/MyDrive/GoogleDataStudio

[Errno 2] No such file or directory: '/content/drive/MyDrive/GoogleDataStudio'
/content


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#!ls

In [11]:
import pandas as pd
import numpy as np

# **START CLEAN DATASET**

## **search_click**

In [12]:
search = pd.read_csv('search_click.csv')
print(search.shape)
search.head()

FileNotFoundError: ignored

In [ ]:
search.info()

> ### **Check Missing Value**

In [ ]:
search.isnull().any()  # True มีค่า missing / False ไม่มีค่า missing

In [ ]:
## --** ต้องใช้ business_id เป็น Primary Key สำหรับ merged กับตารางหลัก 
search2 = search.dropna(subset=['business_id']).reset_index(drop=True)   
## --** ใช้ search_id ต้องใช้เป็น Foreign Key สำหรับ merged ตารางย่อย 
search3 = search2.dropna(subset=['search_id']).reset_index(drop=True)
##ถ้ามี Missing ก็ทำไม่ได้ จะทำให้ row นั้นมีฟีลด์ที่หายไปด้วย 
print(f'Shape:{search3.shape}\n')
print('*'*80)
print(search3.isnull().any(), '\n')
print('*'*120)
search3.head()

In [ ]:
###** -- display_position รหัสสถานที่ == 106 #รหัสแต่ละอัน ไม่รู้มีที่ไหนบ้าง  
print(f'จำนวน display position :{len(set(search3.display_position))}')

In [ ]:
search3['display_position'].value_counts

In [ ]:
search3.to_csv('search_click_clean.csv', encoding='utf-8')

> ### **search_request**

In [ ]:
## Read .csv
search_req = pd.read_csv('search_request.csv')
print(search_req.shape,'\n')
print('*'*120)
print(search_req.isnull().any())
print('*'*200)
search_req.head()

In [ ]:
len(set(search_req['search_id'].tolist()))

In [ ]:
### Drop lat, long **--## ถ้าไม่มีก็ทำ map ไม่ได้  *- ไม่สามารถ Visulization แต่ละพื้นที่ได้
search_req2 = search_req.dropna(subset=['lat', 'long','q']).reset_index(drop=True)
print(search_req2.shape)
print('*'*120)
print(search_req2.isnull().any())

In [ ]:
#search_req2['user_id'].tolist()

In [ ]:
## Drop Columns ที่จะไม่ใช้ในการ Visulizastion 
del search_req2['user_id']  ##1.
del search_req2['Unnamed: 0']  ##2. 
print(search_req2.isnull().any())
print('*'*120)
search_req2.head()  ## จำนวน Row == 402309 

> #### Create columns date 

In [ ]:
date_new = []

for i in range(len(search_req2)):
    date = search_req2['search_timestamp'][i]	
    date = date[:10]
    date_new.append(date)

idx = 2  ##**กำหนดแทรกที่ Columns ที่ 2  -- ชื่อ "search_date"  
search_req2.insert(loc=idx, column='search_date', value=date_new)
search_req2.head()

In [ ]:
search_req2.isnull().any()

In [ ]:
search_req2.to_csv('search_request_clean.csv', encoding='utf-8')

### **Merge Dataframe** 
> ***search_request + search_click***


- ใช้ search_id เป็น Key เพื่อเชื่อมตาราง

In [ ]:
# merged_search3 = pd.merge(search_req2, search3, how="outer", on=["search_id", "search_id"])
# print(merged_search3.shape)
# print('*'*120)
# merged_search3.head()

In [ ]:
# merged_search3 = search3.merge(search_req,how='left',left_on='search_id',right_on='search_id')
# merged_search3.head()

In [ ]:
merged_search = search_req2.merge(search3, how='inner', on='search_id')
print(merged_search.shape, '\n')
print('*'*120)
merged_search.head()

In [ ]:
merged_search.isnull().any()

In [ ]:
merged_search.to_csv('df_merged_search.csv', encoding='utf-8')

## **w_restaurant**

In [ ]:
w_restaurant = pd.read_csv('w_restaurant.csv')
print(w_restaurant.shape,'\n')
print('*'*120)
print(w_restaurant.isnull().any())
print('*'*200)
w_restaurant.head()

> #### Columns 
- zipcode
- number_of_chains
- number_of_bookmarks
- parking
- credit_card_accepted
- bookable
- best_of_wongnai
- wifi
- number_of_seats
- music_venue
- hotel_restaurant
- pet_friendly
- delivery

-- จัดการ Missing Value  = 'ไม่ระบุ'

> #### **Print column name with missing values**

In [ ]:
col_mis = w_restaurant.columns[w_restaurant.isnull().any()].tolist()  # column ที่มี missing
print(len(col_mis))
col_mis

In [ ]:
col_mis = ['zipcode',
 'number_of_chains',
 'number_of_bookmarks',
 #'price_range',
 #'avg_price',
 'parking',
 'credit_card_accepted',
 'good_for_groups',
 'good_for_kids',
 'bookable',
 'best_of_wongnai',
 'wifi',
 'number_of_seats',
 'music_venue',
 'hotel_restaurant',
 'pet_friendly',
 'delivery']

for c in col_mis:
  w_restaurant[c] = w_restaurant[c].fillna('ไม่ระบุ')

print('*'*120)
print(w_restaurant.isnull().any())

> #### **Columns ที่มี Value เป็น 0, 1** --> เปลี่ยนเป็น  
- 0 คือ 'no'
- 1 คือ 'yes'

In [ ]:
col_bol = ['credit_card_accepted', 'good_for_groups', 'good_for_kids', 'hotel_restaurant', 'bookable', 'music_venue',
            'pet_friendly', 'delivery']
print(f'จำนวน Columns :{len(col_bol)}')
print('*'* 100)

for c in col_bol:
    print(w_restaurant[c].value_counts())
    print('*'* 100)

In [ ]:
## -- 0 == 'no', 1 == 'yes'

for c in col_bol:
  for i in range(len(w_restaurant)):
      if w_restaurant[c][i] == 0:
        w_restaurant[c][i] = 'no'
      elif w_restaurant[c][i] == 1:
          w_restaurant[c][i] = 'yes'
    
  print(w_restaurant[c].value_counts())
  print('*'* 100)

> #### **Columns 'parking'**

ที่จอดรถ
- 0 คือ ไม่มีที่จอดรถ
- 1 คือ จอดข้างทาง
- 2 คือ valet
- 3 คือ มีที่จอดรถ



In [ ]:
dict_st = {0:'ไม่มีที่จอดรถ', 1:'จอดรถข้างทาง', 2:'valet', 3:'มีที่จอดรถ' ,'ไม่ระบุ':'ไม่ระบุ'}

for i in range(len(w_restaurant)):
    parking = w_restaurant['parking'][i]
    w_restaurant['parking'][i] = dict_st[parking]

print('*'* 100)
print(w_restaurant['parking'].value_counts())  

> #### **Columns 'best_of_wongnai'**

สถานะ users' choice
- 0 คือ ไม่เป็นร้าน users' choice
- 2018 คือ เป็นร้าน users' choice ปี 2018




In [ ]:
for i in range(len(w_restaurant)):
    if w_restaurant['best_of_wongnai'][i] == 2018:
       w_restaurant['best_of_wongnai'][i] = 'เป็นร้าน users choice ปี 2018'
    elif w_restaurant['best_of_wongnai'][i] == 0:
         w_restaurant['best_of_wongnai'][i] = 'ไม่เป็นร้าน users choice'

print('*'* 100)
print(w_restaurant['best_of_wongnai'].value_counts())

> #### **Columns 'wifi'**

การให้บริการ Wi-Fi
- 0 คือ ไม่มี Wi-Fi บริการ
- 1 คือ มี Wi-Fi บริการฟรี
- 2 คือ มี Wi-Fi บริการ





In [ ]:
dict_st = {0:'ไม่มี Wi-Fi บริการ', 1:'Wi-Fi บริการฟรี', 2:'มี Wi-Fi บริการ', 'ไม่ระบุ':'ไม่ระบุ'}

for i in range(len(w_restaurant)):
    wifi = w_restaurant['wifi'][i]
    w_restaurant['wifi'][i] = dict_st[wifi]

print('*'* 100)
print(w_restaurant['wifi'].value_counts())  

> #### **Columns 'number_of_seats'**

การให้บริการ Wi-Fi
จำนวนที่นั่ง
- 0 คือ ไม่มีที่นั่ง/จัดส่งเท่านั้น
- 1 คือ ไม่เกิน 10 ที่นั่ง
- 2 คือ 11-40 ที่นั่ง
- 3 คือ 41-80 ที่นั่ง
- 4 คือ 81-150 ที่นั่ง
- 5 คือ มากกว่า 150 ที่นั่ง
- 11 คือ 1-10 ที่นั่ง
- 12 คือ 11-25 ที่นั่ง
- 13 คือ 26-50 ที่นั่ง
- 14 คือ มากกว่า 50 ที่นั่ง






In [ ]:
dict_st = {0:'ไม่มีที่นั่ง/จัดส่งเท่านั้น', 1:'ไม่เกิน 10 ที่นั่ง', 2:'11-40 ที่นั่ง', 3:'41-80 ที่นั่ง', 4:' 81-150 ที่นั่ง',
              5:' มากกว่า 150 ที่นั่ง', 11:'1-10 ที่นั่ง', 12:'11-25 ที่นั่ง', 13:'26-50 ที่นั่ง', 14: 'มากกว่า 50 ที่นั่ง', 
                'ไม่ระบุ':'ไม่ระบุ'}

for i in range(len(w_restaurant)):
    #w_restaurant['number_of_seats'][i] = dict_sr[w_restaurant['number_of_seats'][i]]
    nofs = w_restaurant['number_of_seats'][i]
    w_restaurant['number_of_seats'][i] = dict_st[nofs]

print('*'* 100)
print(w_restaurant['number_of_seats'].value_counts())  

> #### **Columns 'domain_id'**

กลุ่มธุรกิจ
- 1 คือ ร้านอาหาร
- 2 คือ ร้านเสริมสวย
- 3 คือ ที่พัก
- 4 คือ สถานที่ท่องเที่ยว






In [ ]:
dict_st = {1:'ร้านอาหาร', 2:'ร้านเสริมสวย', 3:'ที่พัก', 4:'สถานที่ท่องเที่ยว'}

for i in range(len(w_restaurant)):
    mid = w_restaurant['domain_id'][i]
    w_restaurant['domain_id'][i] = dict_st[mid]

print('*'* 100)
print(w_restaurant['domain_id'].value_counts())  

> #### **Columns 'closed'**

สถานะการให้บริการของธุรกิจ
- 0 คือ ร้านเปิดให้บริการอยู่
- 1 คือ ร้านปิดให้บริการแล้ว (permanently closed)






In [ ]:
dict_st = {0:'เปิดให้บริการอยู่', 1:'ปิดให้บริการแล้ว (permanently closed)'}

for i in range(len(w_restaurant)):
    close = w_restaurant['closed'][i]
    w_restaurant['closed'][i] = dict_st[close]

print('*'* 100)
print(w_restaurant['closed'].value_counts())  

> #### **Columns 'new_open'**

สถานะเป็นร้านเปิดใหม่
- 0 คือ ไม่เป็นร้านเปิดใหม่
- 1 คือ เป็นร้านเปิดใหม่






In [ ]:
dict_st = {0:'ไม่เป็นร้านเปิดใหม่', 1:'เป็นร้านเปิดใหม่'}

for i in range(len(w_restaurant)):
    new_open = w_restaurant['new_open'][i]
    w_restaurant['new_open'][i] = dict_st[new_open]

print('*'* 100)
print(w_restaurant['new_open'].value_counts())  

> #### **Columns 'avg_price'** 
- ราคาเฉลี่ย (บาท)

In [ ]:
def groupMeanValue(group):
    w_restaurant['avg_price'] = w_restaurant['avg_price'].fillna(w_restaurant['avg_price'].mean())
    return group

In [ ]:
w_restaurant.groupby("city_name").transform(groupMeanValue).inplace=True

In [ ]:
print(w_restaurant['avg_price'].isnull().any())

In [ ]:
#w_restaurant['avg_price'].tolist()

In [ ]:
# df_city = w_restaurant[['city_name', 'avg_price']]
# df_city0 = df_city.dropna(subset=['avg_price']).reset_index(drop=True)   
# print(df_city0.shape)
# df_city0.head()

In [ ]:
# lst_city = list(set(df_city0['city_name']))
# print(len(lst_city))
# lst_city

In [ ]:
# lst_mean = list()
# for c in lst_city:
#     city_i = df_city0[df_city0['city_name'] == c]
#     m_city_i = city_i['avg_price'].mean()
#     lst_mean.append(m_city_i)

# print(len(lst_mean))
# lst_mean

> #### **Columns 'price_range'** 
ช่วงราคา
- 1 คือ ต่ำกว่า 100 บาท
- 2 คือ 101-250 บาท
- 3 คือ 251-500 บาท
- 4 คือ 501-1000 บาท
- 5 คือ มากกว่า 1000 บาท


In [ ]:
def groupMeanValue2(group):
    w_restaurant['price_range'] = w_restaurant['price_range'].fillna(w_restaurant['price_range'].median())
    return group

In [ ]:
w_restaurant.groupby("city_name").transform(groupMeanValue2).inplace=True

In [ ]:
print(w_restaurant['price_range'].isnull().any())

In [ ]:
print('*'*120)
print(w_restaurant.isnull().any())

### ***search_request + search_click + w_rastaurant***

In [ ]:
w_restaurant2 = w_restaurant.reset_index(drop=True)

In [ ]:
w_restaurant2.to_csv('w_restaurant_clean.csv', encoding='utf-8')

### **Merge Dataframe** 
> ***search_request + (search_click + w_rastaurant)***

- ใช้ business_id เป็น Key เพื่อเชื่อมตาราง


In [ ]:
merged_wongnai = merged_search.merge(w_restaurant2,how='left',left_on='business_id',right_on='id')
print(merged_wongnai.shape, '\n')
print('*'*120)
merged_wongnai.head()

In [ ]:
merged_wongnai.isnull().any()

In [ ]:
merged_wongnai.to_csv('df_wongnai.csv', encoding='utf-8')